In [80]:
# importamos las cosas que vamos a usar de aima
#Xukai Chen; Mario Daniel Gallardo G09
from search import *
import itertools
import math
class ProblemaAscensores(Problem):
    def __init__(self, initial, goal):
        self.goal = goal
        self.initial = initial
        self.analizados = 0
        Problem.__init__(self, initial, goal)
        #initial = ( (pisos de las personas),(0el piso que situa ascensor, 1range inicio de ascensor, 2range final de ascensor,
        #            3booleano de que si lleva persona, 4posicion de persona en la lista de personas, 5destino piso a llegar),... los demas ascensores )
        
    def goal_test(self,estado): 
        self.analizados += 1
        if (estado[0] == self.goal):
            j = False
            for i in estado[1:]:
                if i[3]==1:
                    j = j or True
            if j == False:
                return True
        return False
    
    def listaDeNoLlegados(self, estado):
        # lista de las personas que no han llegado a su destino y que no estén en un ascensor
        lista_final = list()
        lista_de_pos_nollegados = list()
        lista_de_pers_en_asc = list()
        for i in range(len(estado[1:])):
            if estado[i+1][3] == 1:
                lista_de_pers_en_asc.append(estado[i+1][4])        
        for i in range(len(estado[0])):
            if estado[0][i] != self.goal[i] and i not in lista_de_pers_en_asc:
                lista_de_pos_nollegados.append(i)
        lista_de_nollegados = [estado[0][i] for i in lista_de_pos_nollegados]
        lista_final.append(lista_de_nollegados)
        lista_final.append(lista_de_pos_nollegados)
        return lista_final
    
    def persAEntrar(self, asc, estado):
        #dentro de la lista devuelta por fun(listaDeNoLlegados) si hay alguien que cumpla la condicion
        #para subirse al ascensor
        lista_de_nollegados = self.listaDeNoLlegados(estado)
        despl = 0
        while asc[0] in lista_de_nollegados[0][despl:]:
            pos = lista_de_nollegados[0][despl:].index(asc[0])
            destino = self.goal[lista_de_nollegados[1][pos]]
            if (asc[0] < asc[2] and asc[0] < destino) or (asc[0] > asc[1] and asc[0] > destino):
                return True
            despl += 1 + pos
        return False
    def actions(self, estado):
        acs = list()
        accs = list()
        
        lista_de_nollegados = self.listaDeNoLlegados(estado)
        for asc in estado[1:]:
            accaux = list()
            if asc[3] == 0:
                if self.persAEntrar(asc, estado):
                    accaux.append("entrar")
                else:
                    pos_por_encima = list(filter(lambda x: (x >asc[0] and x <= asc[2]), lista_de_nollegados[0]))
                    pos_por_abajo = list(filter(lambda x: (x <asc[0] and x >= asc[1]), lista_de_nollegados[0]))
                    if len(pos_por_encima)>0:
                        accaux.append("subir")
                    if len(pos_por_abajo)>0:
                        accaux.append("bajar")
                    if len(pos_por_encima) == 0 and len(pos_por_abajo) == 0:
                        accaux.append("nada")
            elif asc[3] == 1:
                if asc[0] == asc[5] or (asc[5]> asc[2] and asc[0] == asc[2]) or (asc[5] < asc[1] and asc[0] == asc[1]):
                    accaux.append("salir")
                 
                elif asc[0] < asc[5] and asc[0] < asc[2]:
                    accaux.append("subir")
                elif asc[0] > asc[5] and asc[0] > asc[1]:
                    accaux.append("bajar")
            accs.append(accaux)
            del accaux
        for i in itertools.product(*accs):
            acs.append(i)
        return acs
    
    def result(self, estado, a):
        tup = list()
        list_estado = [list(i) for i in estado]
        idc = 1
        for acc in a:
            asc = list(estado[idc])
            if acc == "entrar":
                lista_de_nollegados = self.listaDeNoLlegados(list_estado)
                if asc[0] in lista_de_nollegados[0]:
                    despl=0
                    while asc[0] in lista_de_nollegados[0][despl:]:
                        pers = lista_de_nollegados[0][despl:].index(asc[0])
                        pos = lista_de_nollegados[1][despl:][pers]
                        despl += pers + 1
                        if (asc[0] < asc[2] and asc[0]<self.goal[pos]) or (asc[0] > asc[1] and asc[0]>self.goal[pos]):
                            asc[3] = 1
                            asc[4] = pos
                            asc[5] = self.goal[pos]
                            break
            elif acc == "salir":
                asc[3] = 0
            elif acc == "subir":
                asc[0] += 1
                if asc[3] == 1:
                    pos_pers = asc[4]
                    list_estado[0][pos_pers] = asc[0]
            elif acc == "bajar":
                asc[0] -= 1
                if asc[3] == 1:
                    pos_pers = asc[4]
                    list_estado[0][pos_pers] = asc[0]
            list_estado[idc] = asc
            del asc
            idc += 1
        for i in list_estado:
            aux = tuple(i)
            tup.append(aux)
        return tuple(tup)


In [71]:
#ascensores = ProblemaAscensores(((2,6,9),(0,0,7,0,0,0),(9,7,12,0,0,0)),(0,12,7)) 
#breadth_first_graph_search(ascensores).solution()
def h1(node):
    h = 0
    estado = node.state
    goal = (3,11,12,1,9)
    for i in range(len(estado[0])):
        h += abs(estado[0][i] - goal[i])
    return h
def llevaPersona(estado):
    for i in estado[1:]:
        if i[3] == 1:
            return True
    return False
def h2(node):
    h = 0
    goal = (3,11,12,1,9)
    estado = node.state
    if llevaPersona == False:
        lista = [x[3] for x in estado[1:]]
        h += min([abs(x-y) for x in estado[0] for y in lista])
    #se divide entre 5 porque la longitud del rango de ascensor es de 5
    aux1 = sum(map(lambda x,y: 2*(math.ceil(abs(x-y)/5)), estado[0],goal))
    h += h1(node)/len(estado[1:]) + aux1
    return h


In [72]:
def resuelve_ascensores(estado_inicial, algoritmo, h=None):

    ascensores=ProblemaAscensores(estado_inicial[0],estado_inicial[1])
    if h: 
        sol= algoritmo(ascensores,h).solution()
    else: 
        sol= algoritmo(ascensores).solution()
    print("Solución: {0}".format(sol))
    print("Algoritmo: {0}".format(algoritmo.__name__))
    if h: 
        print("Heurística: {0}".format(h.__name__))
    else:
        pass
    print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol),ascensores.analizados))

In [81]:
%%time
problema = ProblemaAscensores(((2,4,1,8,1),(0,0,4,0,0,0),(5,4,8,0,0,0),(5,4,8,0,0,0),(9,8,12,0,0,0)),(3,11,12,1,9))

Wall time: 0 ns


In [82]:
E1 = (((2,4,1,8,1),(0,0,4,0,0,0),(5,4,8,0,0,0),(5,4,8,0,0,0),(9,8,12,0,0,0)),(3,11,12,1,9))

In [83]:
%%time
resuelve_ascensores(E1,depth_first_graph_search)

Solución: [('subir', 'bajar', 'bajar', 'bajar'), ('entrar', 'entrar', 'entrar', 'nada'), ('subir', 'subir', 'subir', 'nada'), ('subir', 'subir', 'subir', 'nada'), ('subir', 'subir', 'subir', 'nada'), ('salir', 'subir', 'subir', 'nada'), ('bajar', 'salir', 'entrar', 'nada'), ('bajar', 'bajar', 'bajar', 'entrar'), ('entrar', 'bajar', 'bajar', 'subir'), ('subir', 'bajar', 'bajar', 'subir'), ('salir', 'bajar', 'bajar', 'subir'), ('bajar', 'entrar', 'salir', 'salir'), ('bajar', 'subir', 'nada', 'nada'), ('entrar', 'subir', 'nada', 'nada'), ('subir', 'subir', 'nada', 'nada'), ('subir', 'subir', 'nada', 'nada'), ('subir', 'salir', 'nada', 'nada'), ('salir', 'bajar', 'subir', 'bajar'), ('entrar', 'bajar', 'bajar', 'bajar'), ('bajar', 'bajar', 'entrar', 'bajar'), ('bajar', 'subir', 'subir', 'entrar'), ('bajar', 'nada', 'subir', 'subir'), ('salir', 'nada', 'subir', 'subir'), ('nada', 'nada', 'subir', 'subir'), ('nada', 'nada', 'salir', 'subir'), ('nada', 'subir', 'nada', 'salir'), ('nada', 'subi

In [84]:
%%time
resuelve_ascensores(E1,breadth_first_graph_search)

Solución: [('subir', 'subir', 'subir', 'bajar'), ('entrar', 'subir', 'subir', 'nada'), ('subir', 'subir', 'bajar', 'nada'), ('subir', 'entrar', 'bajar', 'nada'), ('subir', 'bajar', 'bajar', 'nada'), ('salir', 'bajar', 'entrar', 'nada'), ('bajar', 'bajar', 'subir', 'nada'), ('bajar', 'bajar', 'subir', 'nada'), ('entrar', 'salir', 'subir', 'nada'), ('subir', 'entrar', 'subir', 'nada'), ('salir', 'subir', 'salir', 'nada'), ('bajar', 'subir', 'bajar', 'entrar'), ('bajar', 'subir', 'bajar', 'subir'), ('entrar', 'subir', 'bajar', 'subir'), ('subir', 'salir', 'bajar', 'subir'), ('subir', 'bajar', 'subir', 'salir'), ('subir', 'subir', 'subir', 'bajar'), ('salir', 'bajar', 'bajar', 'bajar'), ('entrar', 'subir', 'bajar', 'bajar'), ('bajar', 'bajar', 'entrar', 'entrar'), ('bajar', 'nada', 'subir', 'subir'), ('bajar', 'nada', 'subir', 'subir'), ('salir', 'nada', 'subir', 'subir'), ('nada', 'nada', 'subir', 'subir'), ('nada', 'nada', 'salir', 'salir'), ('nada', 'subir', 'nada', 'bajar'), ('nada', '

In [85]:
%%time
resuelve_ascensores(E1,uniform_cost_search)

Solución: [('subir', 'bajar', 'subir', 'bajar'), ('entrar', 'entrar', 'subir', 'nada'), ('subir', 'subir', 'subir', 'nada'), ('subir', 'subir', 'entrar', 'nada'), ('subir', 'subir', 'bajar', 'nada'), ('salir', 'subir', 'bajar', 'nada'), ('bajar', 'salir', 'bajar', 'nada'), ('bajar', 'bajar', 'bajar', 'entrar'), ('entrar', 'bajar', 'salir', 'subir'), ('subir', 'bajar', 'entrar', 'subir'), ('salir', 'bajar', 'subir', 'subir'), ('bajar', 'nada', 'subir', 'salir'), ('bajar', 'nada', 'subir', 'nada'), ('entrar', 'nada', 'subir', 'nada'), ('subir', 'nada', 'salir', 'nada'), ('subir', 'subir', 'bajar', 'bajar'), ('subir', 'bajar', 'bajar', 'bajar'), ('salir', 'subir', 'bajar', 'bajar'), ('entrar', 'subir', 'bajar', 'entrar'), ('bajar', 'bajar', 'entrar', 'subir'), ('bajar', 'nada', 'subir', 'subir'), ('bajar', 'nada', 'subir', 'subir'), ('salir', 'nada', 'subir', 'subir'), ('nada', 'nada', 'subir', 'salir'), ('nada', 'nada', 'salir', 'nada'), ('nada', 'subir', 'nada', 'bajar'), ('nada', 'subi

In [86]:
%%time
resuelve_ascensores(E1,astar_search, h1)

Solución: [('subir', 'bajar', 'bajar', 'bajar'), ('entrar', 'entrar', 'entrar', 'nada'), ('subir', 'subir', 'subir', 'nada'), ('subir', 'subir', 'subir', 'nada'), ('subir', 'subir', 'subir', 'nada'), ('salir', 'subir', 'subir', 'nada'), ('bajar', 'salir', 'entrar', 'nada'), ('bajar', 'bajar', 'bajar', 'entrar'), ('entrar', 'bajar', 'bajar', 'subir'), ('subir', 'bajar', 'bajar', 'subir'), ('salir', 'bajar', 'bajar', 'subir'), ('bajar', 'entrar', 'salir', 'salir'), ('bajar', 'subir', 'nada', 'nada'), ('entrar', 'subir', 'nada', 'nada'), ('subir', 'subir', 'nada', 'nada'), ('subir', 'subir', 'nada', 'nada'), ('subir', 'salir', 'nada', 'nada'), ('salir', 'bajar', 'subir', 'bajar'), ('entrar', 'bajar', 'bajar', 'bajar'), ('bajar', 'bajar', 'entrar', 'bajar'), ('bajar', 'subir', 'subir', 'entrar'), ('bajar', 'nada', 'subir', 'subir'), ('salir', 'nada', 'subir', 'subir'), ('nada', 'nada', 'subir', 'subir'), ('nada', 'nada', 'salir', 'subir'), ('nada', 'subir', 'nada', 'salir'), ('nada', 'subi

In [87]:
%%time
resuelve_ascensores(E1,astar_search, h2)

Solución: [('subir', 'bajar', 'subir', 'bajar'), ('entrar', 'entrar', 'subir', 'nada'), ('subir', 'subir', 'subir', 'nada'), ('subir', 'subir', 'entrar', 'nada'), ('subir', 'subir', 'bajar', 'nada'), ('salir', 'subir', 'bajar', 'nada'), ('bajar', 'salir', 'bajar', 'nada'), ('bajar', 'bajar', 'bajar', 'entrar'), ('entrar', 'bajar', 'salir', 'subir'), ('subir', 'bajar', 'entrar', 'subir'), ('salir', 'bajar', 'subir', 'subir'), ('bajar', 'nada', 'subir', 'salir'), ('bajar', 'nada', 'subir', 'nada'), ('entrar', 'nada', 'subir', 'nada'), ('subir', 'nada', 'salir', 'nada'), ('subir', 'subir', 'bajar', 'bajar'), ('subir', 'bajar', 'bajar', 'bajar'), ('salir', 'subir', 'bajar', 'bajar'), ('entrar', 'subir', 'bajar', 'entrar'), ('bajar', 'bajar', 'entrar', 'subir'), ('bajar', 'nada', 'subir', 'subir'), ('bajar', 'nada', 'subir', 'subir'), ('salir', 'nada', 'subir', 'subir'), ('nada', 'nada', 'subir', 'salir'), ('nada', 'nada', 'salir', 'nada'), ('nada', 'subir', 'nada', 'bajar'), ('nada', 'subi

In [88]:
%%time
resuelve_ascensores(E1,best_first_graph_search, h2)

Solución: [('subir', 'bajar', 'bajar', 'bajar'), ('entrar', 'entrar', 'entrar', 'nada'), ('subir', 'subir', 'subir', 'nada'), ('subir', 'subir', 'subir', 'nada'), ('subir', 'subir', 'subir', 'nada'), ('salir', 'subir', 'subir', 'nada'), ('bajar', 'salir', 'entrar', 'nada'), ('bajar', 'bajar', 'bajar', 'entrar'), ('entrar', 'bajar', 'bajar', 'subir'), ('subir', 'bajar', 'bajar', 'subir'), ('salir', 'bajar', 'bajar', 'subir'), ('bajar', 'entrar', 'salir', 'salir'), ('bajar', 'subir', 'nada', 'nada'), ('entrar', 'subir', 'nada', 'nada'), ('subir', 'subir', 'nada', 'nada'), ('subir', 'subir', 'nada', 'nada'), ('subir', 'salir', 'nada', 'nada'), ('salir', 'bajar', 'subir', 'bajar'), ('entrar', 'bajar', 'bajar', 'bajar'), ('bajar', 'bajar', 'entrar', 'bajar'), ('bajar', 'subir', 'subir', 'entrar'), ('bajar', 'nada', 'subir', 'subir'), ('salir', 'nada', 'subir', 'subir'), ('nada', 'nada', 'subir', 'subir'), ('nada', 'nada', 'salir', 'subir'), ('nada', 'subir', 'nada', 'salir'), ('nada', 'subi

In [89]:
%%time
resuelve_ascensores(E1,best_first_graph_search, h1)

Solución: [('subir', 'bajar', 'bajar', 'bajar'), ('entrar', 'entrar', 'entrar', 'nada'), ('subir', 'subir', 'subir', 'nada'), ('subir', 'subir', 'subir', 'nada'), ('subir', 'subir', 'subir', 'nada'), ('salir', 'subir', 'subir', 'nada'), ('bajar', 'salir', 'entrar', 'nada'), ('bajar', 'bajar', 'bajar', 'entrar'), ('entrar', 'bajar', 'bajar', 'subir'), ('subir', 'bajar', 'bajar', 'subir'), ('salir', 'bajar', 'bajar', 'subir'), ('bajar', 'entrar', 'salir', 'salir'), ('bajar', 'subir', 'nada', 'nada'), ('entrar', 'subir', 'nada', 'nada'), ('subir', 'subir', 'nada', 'nada'), ('subir', 'subir', 'nada', 'nada'), ('subir', 'salir', 'nada', 'nada'), ('salir', 'bajar', 'subir', 'bajar'), ('entrar', 'bajar', 'bajar', 'bajar'), ('bajar', 'bajar', 'entrar', 'bajar'), ('bajar', 'subir', 'subir', 'entrar'), ('bajar', 'nada', 'subir', 'subir'), ('salir', 'nada', 'subir', 'subir'), ('nada', 'nada', 'subir', 'subir'), ('nada', 'nada', 'salir', 'subir'), ('nada', 'subir', 'nada', 'salir'), ('nada', 'subi